# Applied Data Science Capstone Project
### Singapore: Which areas are underserved by restaurants?



## Table of Contents

1. Introduction / Business Problem
2. Data

## 1. Introduction / Business Problem

Singapore is a city state that is divided into “Planning Areas”. Some areas, particularly the city center, have a high concentration of restaurants that are able to not only draw patrons from nearby residences, but also attract patrons who travel from other areas perhaps because of their established food and service quality.

When opening a new restaurant, it will be sensible to consider areas which are relatively underserved by restaurants, at least so as to minimize competitive pressure for patrons who live nearby.


## 2. Data

A list of Singapore’s Planning Areas will be retrieved from Wikipedia, together with the population count that area. Population is important since areas with higher populations is expected to have a greater demand for restaurants. https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore

The Foursquare API will be used to retrieve lists of restaurants by area, which will then be converted to restaurant counts.

From the above data, a restaurant per population indicator will be calculated for each area. Areas with low restaurants per population would imply that they can support additional restaurants.
